In [1]:
import requests, json, os, sys, datetime, time
from bs4 import BeautifulSoup
import hjson

In [2]:
url = f"https://delhishelterboard.in/maps/jjc-jsc/jjc-new.php"

In [3]:
s = requests.Session() 
r1= s.get(url)
time.sleep(3)
soup1 = BeautifulSoup( r1.text , 'lxml')

### getting the list from select dropdown

In [4]:
listHolder = soup1.select_one('select')

In [5]:
list1 = []
for optHolder in listHolder.select('option'):
    list1.append({'name': optHolder.text.strip(), 'val': int(optHolder['value'])} )
list1

[{'name': 'JJC Code 1', 'val': 201},
 {'name': 'JJC Code 2', 'val': 202},
 {'name': 'JJC Code 3', 'val': 672},
 {'name': 'JJC Code 4', 'val': 203},
 {'name': 'JJC Code 5', 'val': 38},
 {'name': 'JJC Code 6', 'val': 39},
 {'name': 'JJC Code 7', 'val': 204},
 {'name': 'JJC Code 8', 'val': 41},
 {'name': 'JJC Code 9', 'val': 205},
 {'name': 'JJC Code 10', 'val': 43},
 {'name': 'JJC Code 11', 'val': 44},
 {'name': 'JJC Code 12', 'val': 40},
 {'name': 'JJC Code 13', 'val': 597},
 {'name': 'JJC Code 14', 'val': 206},
 {'name': 'JJC Code 15', 'val': 603},
 {'name': 'JJC Code 16', 'val': 42},
 {'name': 'JJC Code 17', 'val': 604},
 {'name': 'JJC Code 18', 'val': 207},
 {'name': 'JJC Code 19', 'val': 79},
 {'name': 'JJC Code 20', 'val': 80},
 {'name': 'JJC Code 21', 'val': 81},
 {'name': 'JJC Code 22', 'val': 82},
 {'name': 'JJC Code 23', 'val': 87},
 {'name': 'JJC Code 24', 'val': 83},
 {'name': 'JJC Code 25', 'val': 88},
 {'name': 'JJC Code 26', 'val': 84},
 {'name': 'JJC Code 27', 'val': 85},

### now starting to scrape each item from the list

In [6]:
geojson = {"type": "FeatureCollection",
    "features": []
}

In [9]:
def fetchShape(p):
    payload=f"jjcid={p['val']}"
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
    }
    try:
        r2 = requests.post(url, headers=headers, data=payload)
    except:
        print("stopped at",p)
        return False
    soup2 = BeautifulSoup( r2.text , 'lxml')
    polyHolder = soup2.select_one('script').contents[0].split('var triangleCoords = ')[1].split(';')[0].replace('\n','')
    polyHolder2 = hjson.loads(polyHolder)
    coords=[]
    for item in polyHolder2:
        lat = item['lat']
        lon = item['lng']
        coords.append([lon,lat])
    
    propHolder1 = soup2.select('div')[1].text.split('\n')
    properties = {}
    for l in propHolder1:
        l1 = l.strip()
        if len(l1):
            l2 = l1.split(' : ')
            if len(l2) >= 2:
                properties[l2[0]] = l2[1]
            else:
                print("Strange property:",l1)
    feature = {"type": "Feature",
        "properties": properties,
        "geometry": {
            "type": "Polygon",
            "coordinates": [coords]
        }
    }
    return feature

### due to occasional timeouts, putting into a while loop to keep trying after a pause in case of failure

In [8]:
for N,p in enumerate(list1):
    print(N,p)
    feature = False
    while(not feature):
        feature = fetchShape(p)
        if not feature:
            timeout = 10
            print(f"Timeout, waiting {timeout} secs")
            time.sleep(timeout)
        else:
            geojson['features'].append(feature)
            time.sleep(1)
            

0 {'name': 'JJC Code 1', 'val': 201}
1 {'name': 'JJC Code 2', 'val': 202}
2 {'name': 'JJC Code 3', 'val': 672}
3 {'name': 'JJC Code 4', 'val': 203}
4 {'name': 'JJC Code 5', 'val': 38}
5 {'name': 'JJC Code 6', 'val': 39}
6 {'name': 'JJC Code 7', 'val': 204}
7 {'name': 'JJC Code 8', 'val': 41}
8 {'name': 'JJC Code 9', 'val': 205}
9 {'name': 'JJC Code 10', 'val': 43}
10 {'name': 'JJC Code 11', 'val': 44}
11 {'name': 'JJC Code 12', 'val': 40}
12 {'name': 'JJC Code 13', 'val': 597}
13 {'name': 'JJC Code 14', 'val': 206}
14 {'name': 'JJC Code 15', 'val': 603}
15 {'name': 'JJC Code 16', 'val': 42}
16 {'name': 'JJC Code 17', 'val': 604}
17 {'name': 'JJC Code 18', 'val': 207}
18 {'name': 'JJC Code 19', 'val': 79}
19 {'name': 'JJC Code 20', 'val': 80}
20 {'name': 'JJC Code 21', 'val': 81}
21 {'name': 'JJC Code 22', 'val': 82}
22 {'name': 'JJC Code 23', 'val': 87}
23 {'name': 'JJC Code 24', 'val': 83}
24 {'name': 'JJC Code 25', 'val': 88}
25 {'name': 'JJC Code 26', 'val': 84}
26 {'name': 'JJC Cod

153 {'name': 'JJC Code 157', 'val': 507}
stopped at {'name': 'JJC Code 157', 'val': 507}
Timeout, waiting 10 secs
stopped at {'name': 'JJC Code 157', 'val': 507}
Timeout, waiting 10 secs
stopped at {'name': 'JJC Code 157', 'val': 507}
Timeout, waiting 10 secs
stopped at {'name': 'JJC Code 157', 'val': 507}
Timeout, waiting 10 secs
stopped at {'name': 'JJC Code 157', 'val': 507}
Timeout, waiting 10 secs
stopped at {'name': 'JJC Code 157', 'val': 507}
Timeout, waiting 10 secs
154 {'name': 'JJC Code 158', 'val': 518}
155 {'name': 'JJC Code 159', 'val': 519}
156 {'name': 'JJC Code 160', 'val': 517}
157 {'name': 'JJC Code 161', 'val': 516}
158 {'name': 'JJC Code 162', 'val': 501}
159 {'name': 'JJC Code 163', 'val': 502}
160 {'name': 'JJC Code 164', 'val': 450}
161 {'name': 'JJC Code 165', 'val': 482}
162 {'name': 'JJC Code 166', 'val': 490}
163 {'name': 'JJC Code 167', 'val': 505}
164 {'name': 'JJC Code 168', 'val': 433}
165 {'name': 'JJC Code 169', 'val': 493}
166 {'name': 'JJC Code 170', 

290 {'name': 'JJC Code 297', 'val': 181}
291 {'name': 'JJC Code 298', 'val': 180}
292 {'name': 'JJC Code 299', 'val': 182}
293 {'name': 'JJC Code 300', 'val': 185}
294 {'name': 'JJC Code 301', 'val': 184}
295 {'name': 'JJC Code 302', 'val': 183}
296 {'name': 'JJC Code 303', 'val': 276}
297 {'name': 'JJC Code 304', 'val': 280}
298 {'name': 'JJC Code 305', 'val': 281}
299 {'name': 'JJC Code 306', 'val': 381}
300 {'name': 'JJC Code 307', 'val': 382}
301 {'name': 'JJC Code 309', 'val': 383}
302 {'name': 'JJC Code 310', 'val': 384}
303 {'name': 'JJC Code 311', 'val': 626}
304 {'name': 'JJC Code 312', 'val': 376}
stopped at {'name': 'JJC Code 312', 'val': 376}
Timeout, waiting 10 secs
stopped at {'name': 'JJC Code 312', 'val': 376}
Timeout, waiting 10 secs
stopped at {'name': 'JJC Code 312', 'val': 376}
Timeout, waiting 10 secs
stopped at {'name': 'JJC Code 312', 'val': 376}
Timeout, waiting 10 secs
stopped at {'name': 'JJC Code 312', 'val': 376}
Timeout, waiting 10 secs
stopped at {'name': 

stopped at {'name': 'JJC Code 443', 'val': 304}
Timeout, waiting 10 secs
stopped at {'name': 'JJC Code 443', 'val': 304}
Timeout, waiting 10 secs
430 {'name': 'JJC Code 444', 'val': 306}
431 {'name': 'JJC Code 445', 'val': 305}
432 {'name': 'JJC Code 446', 'val': 303}
433 {'name': 'JJC Code 447', 'val': 300}
434 {'name': 'JJC Code 448', 'val': 299}
435 {'name': 'JJC Code 449', 'val': 298}
436 {'name': 'JJC Code 450', 'val': 301}
437 {'name': 'JJC Code 451', 'val': 294}
438 {'name': 'JJC Code 452', 'val': 296}
439 {'name': 'JJC Code 453', 'val': 26}
440 {'name': 'JJC Code 454', 'val': 302}
441 {'name': 'JJC Code 455', 'val': 295}
442 {'name': 'JJC Code 456', 'val': 1}
443 {'name': 'JJC Code 457', 'val': 2}
444 {'name': 'JJC Code 458', 'val': 609}
445 {'name': 'JJC Code 459', 'val': 11}
446 {'name': 'JJC Code 460', 'val': 6}
447 {'name': 'JJC Code 461', 'val': 7}
448 {'name': 'JJC Code 462', 'val': 9}
449 {'name': 'JJC Code 463', 'val': 4}
450 {'name': 'JJC Code 464', 'val': 14}
451 {'na

574 {'name': 'JJC Code 599', 'val': 345}
575 {'name': 'JJC Code 600', 'val': 430}
576 {'name': 'JJC Code 601', 'val': 641}
577 {'name': 'JJC Code 602', 'val': 637}
578 {'name': 'JJC Code 603', 'val': 638}
579 {'name': 'JJC Code 604', 'val': 639}
580 {'name': 'JJC Code 605', 'val': 644}
581 {'name': 'JJC Code 606', 'val': 643}
582 {'name': 'JJC Code 607', 'val': 642}
583 {'name': 'JJC Code 608', 'val': 640}
stopped at {'name': 'JJC Code 608', 'val': 640}
Timeout, waiting 10 secs
stopped at {'name': 'JJC Code 608', 'val': 640}
Timeout, waiting 10 secs
stopped at {'name': 'JJC Code 608', 'val': 640}
Timeout, waiting 10 secs
stopped at {'name': 'JJC Code 608', 'val': 640}
Timeout, waiting 10 secs
stopped at {'name': 'JJC Code 608', 'val': 640}
Timeout, waiting 10 secs
stopped at {'name': 'JJC Code 608', 'val': 640}
Timeout, waiting 10 secs
584 {'name': 'JJC Code 609', 'val': 90}
585 {'name': 'JJC Code 610', 'val': 91}
586 {'name': 'JJC Code 611', 'val': 537}
587 {'name': 'JJC Code 612', 'v

IndexError: list index out of range

### strange property line encountered, so improving function and resuming

In [10]:
for N,p in enumerate(list1[690:]):
    print(690+N,p)
    feature = False
    while(not feature):
        feature = fetchShape(p)
        if not feature:
            timeout = 10
            print(f"Timeout, waiting {timeout} secs")
            time.sleep(timeout)
        else:
            geojson['features'].append(feature)
            time.sleep(1)
            

690 {'name': 'JJC Code 741', 'val': 747}
Strange property: Tahirpur  (Regarding  Toilets &
Strange property: Water Supply)
691 {'name': 'JJC Code 742', 'val': 746}
Strange property: Dilshad garden
Strange property: Khera
Strange property: (Regarding  Toilets &
Strange property: Water Supply)
692 {'name': 'JJC Code 743', 'val': 683}
693 {'name': 'JJC Code 744', 'val': 712}
694 {'name': 'JJC Code 745', 'val': 713}
695 {'name': 'JJC Code 746', 'val': 710}
696 {'name': 'JJC Code 749', 'val': 711}
697 {'name': 'JJC Code 750', 'val': 681}
698 {'name': 'JJC Code 755', 'val': 718}
699 {'name': 'JJC Code 756', 'val': 715}
700 {'name': 'JJC Code 757', 'val': 716}
701 {'name': 'JJC Code 758', 'val': 717}
702 {'name': 'JJC Code 761', 'val': 679}
703 {'name': 'JJC Code 763', 'val': 677}
704 {'name': 'JJC Code 764', 'val': 678}
705 {'name': 'JJC Code 765', 'val': 682}
706 {'name': 'JJC Code 766', 'val': 680}
707 {'name': 'JJC Code 769', 'val': 714}
708 {'name': 'JJC Code 771', 'val': 728}
709 {'name

In [ ]:
# geojson

In [11]:
json.dump(geojson, open('jj.geojson','w'), indent=2)

### all shapes collected and saved to the .geojson file